In [2]:
!pip install -q transformers datasets peft accelerate bitsandbytes rouge-score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 20.1 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset

dataset = load_dataset("iamtarun/code_instructions_120k_alpaca")

print("Dataset loaded ✅")

Dataset loaded ✅


In [5]:
def filter_python(example):
    return "def " in example["output"] or "import " in example["output"]

filtered = dataset["train"].filter(filter_python)

print("Filtered size:", len(filtered))

Filtered size: 61310


In [6]:
filtered = filtered.shuffle(seed=42).select(range(4000))

# Split into train / validation / test
split = filtered.train_test_split(test_size=0.2, seed=42)
test_valid = split["test"].train_test_split(test_size=0.5, seed=42)

train_dataset = split["train"]
val_dataset = test_valid["train"]
test_dataset = test_valid["test"]

print("Train:", len(train_dataset))
print("Validation:", len(val_dataset))
print("Test:", len(test_dataset))

Train: 3200
Validation: 400
Test: 400


In [7]:
def format_example(example):
    text = f"""Below is an instruction that describes a task.

### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""

    return {"text": text}

train_dataset = train_dataset.map(format_example)
val_dataset = val_dataset.map(format_example)

print("Columns:", train_dataset.column_names)

Columns: ['instruction', 'input', 'output', 'prompt', 'text']


In [8]:
# IMPORT TOKENIZER
from transformers import AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded successfully ✅")

Tokenizer loaded successfully ✅


In [9]:


def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=256,
        padding="max_length"
    )

    # Labels required for training
    tokens["labels"] = tokens["input_ids"].copy()

    return tokens

train_dataset = train_dataset.map(
    tokenize_function,
    remove_columns=train_dataset.column_names
)

val_dataset = val_dataset.map(
    tokenize_function,
    remove_columns=val_dataset.column_names
)

print("Columns:", train_dataset.column_names)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Columns: ['input_ids', 'attention_mask', 'labels']


In [10]:


from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float32
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Set pad token
model.config.pad_token_id = tokenizer.eos_token_id

print("Model loaded with quantization ✅")

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded with quantization ✅


In [11]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for QLoRA training
model = prepare_model_for_kbit_training(model)

# Define QLoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply adapters
model = get_peft_model(model, lora_config)

# Show trainable params
model.print_trainable_parameters()

print("QLoRA adapters applied successfully ✅")

trainable params: 2,252,800 || all params: 1,102,301,184 || trainable%: 0.2044
QLoRA adapters applied successfully ✅


In [12]:


from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qlora-output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    max_steps=100,            # fast training for MSc project
    logging_steps=10,
    save_steps=50,
    save_total_limit=1,
    fp16=False,               # safe setting
    report_to="none",
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

print("Training started...")

trainer.train()

print("Training completed successfully ✅")

Training started...


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1181: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,5.608969
20,4.565918
30,2.861255
40,1.639643
50,1.471886
60,0.963778
70,1.014346
80,0.881252
90,0.838682
100,0.858066


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1181: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Training completed successfully ✅


In [13]:

prompt = """Below is an instruction.

### Instruction:
Write a Python function to add two numbers.

### Input:


### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.7,
    top_p=0.9
)

generated_code = tokenizer.decode(output[0], skip_special_tokens=True)

print("Generated Output:\n")
print(generated_code)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1181: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:232: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  check_backward_validity(args)


Generated Output:

Below is an instruction.

### Instruction:
Write a Python function to add two numbers.

### Input:


### Response:




In [16]:
def add(a, b):
    return a + b

In [19]:
from evaluate import load

bleu = load("bleu")
rouge = load("rouge")

prediction = generated_code

reference = """def add(a, b):
    return a + b"""

bleu_score = bleu.compute(
    predictions=[prediction],
    references=[[reference]]
)

rouge_score = rouge.compute(
    predictions=[prediction],
    references=[reference]
)

print("BLEU Score:", bleu_score)
print("ROUGE Score:", rouge_score)

BLEU Score: {'bleu': 0.0, 'precisions': [0.10344827586206896, 0.0, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.4166666666666665, 'translation_length': 29, 'reference_length': 12}
ROUGE Score: {'rouge1': np.float64(0.18181818181818182), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.09090909090909091), 'rougeLsum': np.float64(0.18181818181818182)}


In [20]:
model.save_pretrained("./qlora-trained-model")
tokenizer.save_pretrained("./qlora-trained-model")

print("QLoRA model saved successfully ✅")

QLoRA model saved successfully ✅
